#### CLeaning training data( removing inconvertible rows of data and converting rest to date time )

In [1]:
import pandas as pd

# Read the data
df = pd.read_csv("stock_prices_training.csv")

# Strip spaces and convert Date Fetched column to datetime (Handles both date and timestamp formats)
df["Date Fetched"] = pd.to_datetime(df["Date Fetched"].str.strip(), errors='coerce')

# Check for any rows where Date Fetched couldn't be converted
if df["Date Fetched"].isna().sum() > 0:
    print("⚠️ Warning: Some dates could not be parsed. Check for inconsistencies.")

# Save cleaned data (optional)
df.to_csv("stock_prices_training.csv", index=False)

# Display the first few rows
print(df.head())

  Stock Date Fetched  Last Price ($)
0  AAPL   2024-03-18      171.470352
1  AAPL   2024-03-25      170.674103
2  AAPL   2024-04-01      168.783020
3  AAPL   2024-04-08      175.720291
4  AAPL   2024-04-15      164.224564


## SARIMA Model 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime, timedelta

# Read the Stock Prices Data
df = pd.read_csv("stock_prices_training.csv", dtype={"Date Fetched": str})  # Read as string first
df["Date Fetched"] = df["Date Fetched"].str.strip()  # Remove any extra spaces
df["Date Fetched"] = pd.to_datetime(df["Date Fetched"], format="%Y-%m-%d")  # Convert to datetime

# Sort by Date (to ensure proper time series order)
df = df.sort_values(by=["Stock", "Date Fetched"]).reset_index(drop=True)

# Get Unique Stocks
unique_stocks = df["Stock"].unique()

# Store Predictions
predictions = []

today = datetime.today().strftime("%Y-%m-%d")  # Get today's date

# Iterate Over Each Stock to Apply SARIMA Model
for stock in unique_stocks:
    stock_data = df[df["Stock"] == stock]
    stock_data = stock_data.set_index("Date Fetched")["Last Price ($)"].dropna()
    
    # Check if there is enough data
    if len(stock_data) < 10:
        print(f"Not enough data for {stock}, skipping...")
        continue
    
    # Fit SARIMA Model (Seasonal Order assumes weekly seasonality)
    try:
        model = SARIMAX(stock_data, order=(1,1,1), seasonal_order=(1,1,1,52), enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False)
        
        # Forecast Today's Price
        forecast = results.forecast(steps=1)
        predicted_price = forecast.iloc[0]
        predictions.append({"Stock": stock, "Predicted Price ($)": round(predicted_price, 2), "Date Predicted": today})
        
        print(f" Predicted Price for {stock} on {today}: ${predicted_price:.2f}")
    except Exception as e:
        print(f" Error processing {stock}: {e}")

# Convert Predictions to DataFrame
pred_df = pd.DataFrame(predictions)
print(pred_df)

# Save Predictions to CSV (Optional)
pred_df.to_csv("stock_price_predictions.csv", index=False)

C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

 Predicted Price for AAPL on 2025-04-06: $195.32


C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

 Predicted Price for AMD on 2025-04-06: $78.62


C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

...

 Predicted Price for WFC on 2025-04-06: $60.07


C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

 Predicted Price for WMT on 2025-04-06: $83.48


C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency W-MON will be used.
  self._init_dates(dates, freq)
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for ARMA and trend. All parameters except for variances will be set to zeros.
  warn('Too few observations to estimate starting parameters%s.'
C:\Users\kshit\AppData\Local\Programs\Python\Python313\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:866: UserWarning: Too few observations to estimate starting parameters for seasonal AR

 Predicted Price for XOM on 2025-04-06: $103.37
    Stock  Predicted Price ($) Date Predicted
0    AAPL               195.32     2025-04-06
1     AMD                78.62     2025-04-06
2    AMZN               172.06     2025-04-06
3    AVGO               146.75     2025-04-06
4      BA               123.00     2025-04-06
5     BAC                33.10     2025-04-06
6       C                56.27     2025-04-06
7   CMCSA                32.19     2025-04-06
8     COP                84.04     2025-04-06
9    COST               933.97     2025-04-06
10    CVX               140.75     2025-04-06
11    DIS                79.20     2025-04-06
12      F                 8.96     2025-04-06
13     GD               242.84     2025-04-06
14     GM                43.04     2025-04-06
15  GOOGL               150.81     2025-04-06
16     GS               452.63     2025-04-06
17     HD               339.26     2025-04-06
18   INTC                16.86     2025-04-06
19    JNJ               148.52  

#### Arranging output in same order as stock universe

In [3]:
# Define the stock order from us_stock_universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Flatten the stock order list
stock_order = [stock for sector in us_stock_universe.values() for stock in sector]

# Load the CSV file
df = pd.read_csv("stock_price_predictions.csv")

# Ensure the stock column is treated as a category with the correct order
df["Stock"] = pd.Categorical(df["Stock"], categories=stock_order, ordered=True)

# Sort the dataframe by the defined stock order
df_sorted = df.sort_values("Stock")

# Save the sorted dataframe
df_sorted.to_csv("stock_price_predictions.csv", index=False)

print(" Data successfully saved in stock_price_predictions.csv in the correct sector and stock order.")


 Data successfully saved in stock_price_predictions.csv in the correct sector and stock order.


## Stock weight Assigning Model

In [4]:
import csv
import os

In [5]:
training_data=pd.read_csv("training_data.csv")

In [6]:
def fetch_row(file_name, row_index):
    with open(file_name, mode='r', newline='') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for idx, row in enumerate(reader):
            if idx == row_index:
                return [float(x) if x.strip() else 0.0 for x in row[2:10]] 
    return None

In [7]:
def Descent_w(x, y, num_rows, w, b, k):
    if num_rows == 0:
        return 0  # Avoid division by zero

    dex_w = 0
    nan_detected = False
    nan_index = -1  # Store the first index where NaN occurs

    for j in range(num_rows):
        term = (np.dot(x[j], w) + b - y[j]) * x[j][k]
        if np.isnan(term):
            nan_detected = True
            nan_index = j  # Store the index of the first NaN value
        dex_w += term

    dex_w /= num_rows  # Average gradient

    # Debugging NaN values
    if nan_detected:  # If any NaN was found inside sum
        print(f"⚠ NaN detected in Descent_w at index {k} (j={nan_index})!")
        print(f"x[{nan_index}]: {x[nan_index]}, y[{nan_index}]: {y[nan_index]}, w: {w}, b: {b}")
        dex_w = 0.0  # Prevent NaN propagation
    return dex_w


In [8]:
def Descent_b(x, y, num_rows, w, b):
    if num_rows == 0:
        return 0  # Avoid division by zero

    dex_b = sum(np.dot(x[j], w) + b - y[j] for j in range(num_rows))
    dex_b /= num_rows  # Average gradient

    # Debugging NaN values
    if np.isnan(dex_b):
        print("NaN detected in Descent_b!")
        dex_b = 0.0  # Prevent NaN propagation

    return dex_b

In [9]:
def Descent(x, y, num_rows, w, b):
    if num_rows == 0:
        return 0  # Avoid division by zero

    dex = (sum(np.dot(x[j], w) + b - y[j] for j in range(num_rows)))**2
    dex /= (num_rows*2)  # Average gradient

    # Debugging NaN values
    if np.isnan(dex):
        print("NaN detected in Descent_b!")
        dex = 0.0  # Prevent NaN propagation

    return dex

In [10]:
b = np.zeros((1))
w = np.zeros(8)  # All weights start from 0
learn_rate=0.001

temp_b=0
temp_w=np.zeros(8)

num_rows = training_data.shape[0]
num_columns=8
x_train=np.zeros((num_rows,num_columns))
y_train=np.zeros(num_rows)

for i in range(num_rows):
    x_train[i] = fetch_row("training_data.csv", i)
y_train = pd.read_csv("training_data.csv")["Weight"]

mean = np.mean(x_train, axis=0)
std = np.std(x_train, axis=0)
std[std == 0] = 1  # Prevent division by zero
x_train = (x_train - mean) / std

for iter in range(3):
    for i in range(num_rows):
        learn_rate = 0.01 / (1 + 0.001 * i)
        for j in range(8):
            des=Descent_w(x_train,y_train,num_rows,w,b,j)
            temp_w[j]=w[j]-learn_rate*des
        des_b=learn_rate*Descent_b(x_train,y_train,num_rows,w,b)
        temp_b=b-des_b
        for j in range(8):
            w[j]=temp_w[j]
        b=temp_b
        print(f"descent = {Descent(x_train,y_train,num_rows,w,b)}")

C:\Users\kshit\AppData\Local\Temp\ipykernel_8628\3083087831.py:27: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  temp_w[j]=w[j]-learn_rate*des


descent = [3503450.84097371]
descent = [3433801.46861561]
descent = [3365604.52788211]
descent = [3298828.31974031]
descent = [3233441.86771136]


descent = [3169414.9007014]
descent = [3106717.83625702]
descent = [3045321.76423427]
descent = [2985198.43087088]
descent = [2926320.22325097]


descent = [2868660.15415251]
descent = [2812191.84726756]
descent = [2756889.52278588]
descent = [2702727.98333247]
descent = [2649682.60025021]


descent = [2597729.30021861]
descent = [2546844.55220027]
descent = [2497005.3547066]
descent = [2448189.22337474]
descent = [2400374.1788478]


...

descent = [6.34149279e-16]
descent = [6.28686629e-16]


descent = [6.23230354e-16]
descent = [6.17871899e-16]


descent = [6.12398083e-16]
descent = [6.07038598e-16]


descent = [6.01767643e-16]
descent = [5.96682721e-16]
descent = [5.9135389e-16]


descent = [5.86264988e-16]


In [11]:
x_test=np.zeros((50,8))
ratio=np.zeros(50)

for i in range(50):
    x_test[i]=fetch_row("testing_data.csv",i)
    ratio[i]=np.dot(x_test[i],w)+b

mean = np.mean(x_test, axis=0)
std = np.std(x_test, axis=0)
std[std == 0] = 1  # Prevent division by zero
x_test = (x_test - mean) / std

final_result={
    "Stock": pd.read_csv("testing_data.csv")["Stock"],
    "Price":pd.read_csv("testing_data.csv")["Price"],
    "Weights":ratio
}
df=pd.DataFrame(final_result)
df.to_csv("results.csv",index=False)
w

C:\Users\kshit\AppData\Local\Temp\ipykernel_8628\378531489.py:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  ratio[i]=np.dot(x_test[i],w)+b


array([  8.7851721 ,  36.18964973, -45.02130807,  -2.28342352,
        -0.38380117,  23.57913542,  28.50323659, -64.74361559])

In [12]:
weight_bias={
    "Weights":w,
    "Bias":[b for i in range(len(w))]
}
dataFrame=pd.DataFrame(weight_bias)
dataFrame.to_csv("Weight_Bias.csv",index=False)

##### Updating Training Data

In [13]:
def create_csv(filename="training_data.csv"):
    columns = ["Sector", "Stock", "Price", "P/E", "Sector Lowest", "52 Week High", "200d-EMA", "Alpha", "Beta", "Worth", "Weight"]
    if not os.path.exists(filename):
        df = pd.DataFrame(columns=columns)
        df.to_csv(filename, index=False)
        print(f"{filename} created successfully.")
    else:
        print(f"{filename} already exists.")

def update_csv(sector, stock, price, pe, sector_lowest, high_52, dema, alpha, beta, worth, weight, filename="training_data.csv"):
    data = {
        "Sector": sector,
        "Stock": stock,
        "Price": price,
        "P/E": pe,
        "Sector Lowest": sector_lowest,
        "52 Week High": high_52,
        "200d-EMA": dema,
        "Alpha": alpha,
        "Beta": beta,
        "Worth": worth,
        "Weight": weight
    }
    
    df = pd.DataFrame(data)
    df.to_csv(filename, mode='a', header=not os.path.exists(filename), index=False)
    print(f"Data appended to {filename} successfully.")

# Example Usage:
# Create the CSV file if not exists
create_csv()

# Example data to update the CSV file
update_csv(
    sector=pd.read_csv("testing_data.csv")["Sector"],
    stock=pd.read_csv("testing_data.csv")["Stock"],
    price=pd.read_csv("testing_data.csv")["Price"],
    pe=pd.read_csv("testing_data.csv")["P/E"],
    sector_lowest=pd.read_csv("testing_data.csv")["Sector Lowest"],
    high_52=pd.read_csv("testing_data.csv")["52 Week High"],
    dema=pd.read_csv("testing_data.csv")["200d-EMA"],
    alpha=pd.read_csv("testing_data.csv")["Alpha"],
    beta=pd.read_csv("testing_data.csv")["Beta"],
    worth=pd.read_csv("testing_data.csv")["Worth"],
    weight=ratio
)

training_data.csv already exists.
Data appended to training_data.csv successfully.


## Picking the 10 Best Stocks

In [14]:
results=pd.read_csv("results.csv")
chosen_index=[-1 for i in range(10)]
liquidity=pd.read_csv("liquidity.csv")["Amihud's Illiquidity Ratio"]
stock_data=pd.read_csv("stock_prices.csv")
stock_sector=stock_data["Sector"]
stock_name=stock_data["Stock"]
stock_price=stock_data["Last Price ($)"]

In [15]:
weights=results["Weights"]
num_row=len(weights)
weight_sum=0

In [16]:
for j in range(10):
    max=-float('inf')
    max_index=-1
    for i in range(num_row):
        if not(i in chosen_index):
            if max<weights[i]:
                max=weights[i]
                max_index=i
    chosen_index[j]=max_index
    weight_sum+=weights[max_index]
chosen_index

[26, 27, 28, 29, 23, 46, 22, 12, 20, 10]

In [17]:
data={
    "Sector":[stock_sector[k] for k in chosen_index],
    "Stock": [stock_name[i] for i in chosen_index],
    "Price": [stock_price[i] for i in chosen_index],
    "Liquidity": [liquidity[i] for i in chosen_index],
    "Weights": [weights[i]*100/weight_sum for i in chosen_index]
}
darf = pd.DataFrame(data)
darf.to_csv("Stock_final_investment.csv",index=False)